In [ ]:
# @title Install requirements
from IPython.display import clear_output
# @markdown We install the acme library, see [here](https://github.com/deepmind/acme) for more info.

# @markdown **WARNING:** There may be *errors* and/or *warnings* reported during the installation. However, they should be ignored.
!pip install --upgrade pip --quiet
!pip install imageio --quiet
!pip install imageio-ffmpeg
!pip install gym --quiet
!pip install enum34 --quiet
!pip install dm-env --quiet
!pip install pandas --quiet
!pip install grpcio==1.34.0 --quiet
!pip install tensorflow --quiet
!pip install typing --quiet
!pip install einops --quiet
!pip install dm-acme --quiet
!pip install dm-acme[reverb] --quiet
!pip install dm-acme[jax,tensorflow] --quiet
!pip install dm-acme[envs] --quiet
!pip install dm-env --quiet

!pip install git+https://github.com/NeuromatchAcademy/evaltools --quiet
from evaltools.airtable import AirtableForm

# Generate airtable form
atform = AirtableForm('appn7VdPRseSoMXEG','W3D2_T1','https://portal.neuromatchacademy.org/api/redirect/to/3e77471d-4de0-4e43-a026-9cfb603b5197')
clear_output()

In [ ]:
#@title Import modules
# Import modules
import gym
import enum
import copy
import time
import acme
import torch
import base64
import dm_env
import IPython
import imageio
import warnings
import itertools
import collections

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import scipy
from scipy.optimize import linprog
from tqdm import tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt

from acme import specs
from acme import wrappers
from acme.utils import tree_utils
from acme.utils import loggers
from torch.autograd import Variable
from torch.distributions import Categorical
from typing import Callable, Sequence

warnings.filterwarnings('ignore')
np.set_printoptions(precision=3, suppress=1)

## Creating test_data 

In [ ]:
#@title test_data
data = {'1':{'F':'2'}, '2':{'F':'3', 'B':'1'}, '3':{'F':'4', 'B':'2', 'T':'23'}, '4':{'B':'3'},
        '21':{'F':'22'}, '22':{'F':'23', 'B':'21'}, '23':{'F':'24', 'B':'21', 'T':'3'}, '24':{'B':'23'}
        }
distance = { ('1','2'):70, ('2','3'):50, ('3','4'):70, ('21','22'):50, ('22','23'):50, ('23','24'):50, ('3','23'):0,
             ('2','1'):70, ('3','2'):50, ('4','3'):70, ('22','21'):50, ('23','21'):50, ('24','23'):50, ('23','3'):0
            }
q = {('1','F'): 0,('2','F'): 0,('2','B'): 0, ('3','F'): 0,('3','B'): 0, ('3','T'): 0,('4','B'): 0,
     ('21','F'): 0,('22','F'): 0,('22','B'): 0, ('23','F'): 0,('23','B'): 0, ('23','T'): 0,('24','B'): 0}


In [ ]:
#@title test_data
data = {'1':{'F':'2','F2':'21'}, '2':{'F':'3', 'B':'1', 'T':'22'}, '3':{'F':'4', 'B':'2', 'T':'23'}, '4':{'B':'3','F':'24'},
        '21':{'F':'22','B2':'1'}, '22':{'F':'23', 'B':'21', 'T':'2'}, '23':{'F':'24', 'B':'22', 'T':'3'}, '24':{'B':'23'}
        }
distance = { ('1','2'):70, ('2','3'):70, ('3','4'):70, ('21','22'):50, ('22','23'):40, ('23','24'):50, ('3','23'):0, ('2','22'):0, ('4','24'):500, ('1','21'):560, 
             ('2','1'):70, ('3','2'):70, ('4','3'):70, ('22','21'):50, ('23','22'):40, ('24','23'):50, ('23','3'):0, ('22','2'):0, ('24','4'):500, ('21','1'):560
}   

In [ ]:
# @title Environment 1
class Environment:
  
  def __init__(self, dt,dc):
    self.x = dt
    self.distance = dc
    self.alpha = [1,1]

  def Possible_actions(self,s):
    p_actions = list(self.x[s].keys())
    self.p_actions = p_actions
    return p_actions
    
  def next_step(self,s,a):
    n_step = self.x[s][a] 
    return n_step
  
  def Reward(self,s,a):
    reward = 0
    t_reward = 0
    # ns = self.n_step
    w1 =1
    w2 =1

    if a == 'T':
      t_reward += -200
    else:
      reward += (-1) * self.distance[(s, self.next_step(s,a))]

    # total_reward = w1 * reward + w2 * t_reward

    r= [reward,t_reward]
    total_reward = np.dot(self.alpha,r)

    if self.next_step(s,a) == '24':
      total_reward += 1000
    
    # total_reward = w1 * reward + w2 * t_reward
    return float(total_reward)
    

  def Reward_features(self,s,a):
    reward = 0
    t_reward = 0
    # ns = self.n_step
    w1 =1
    w2 =1

    if a == 'T':
      t_reward += -200
    else:
      reward += (-1) * self.distance[(s, self.next_step(s,a))]

    return (reward, t_reward)



env = Environment(data,distance)

In [ ]:
# @title Environment 2
class Environment2:
  
  def __init__(self, dt,dc):
    self.x = dt
    self.distance = dc
    self.alpha = [1,1]

  def Possible_actions(self,s):
    p_actions = list(self.x[s].keys())
    self.p_actions = p_actions
    return p_actions
    
  def next_step(self,s,a):
    n_step = self.x[s][a] 
    return n_step
  
  def Reward(self,s,a):
    reward = 0
    t_reward = 0
    # ns = self.n_step
    w1 =1
    w2 =1

    if a == 'T':
      t_reward += -200
    else:
      reward += (-1) * self.distance[(s, self.next_step(s,a))]

    # total_reward = w1 * reward + w2 * t_reward

    r= [reward,t_reward]
    total_reward = np.dot(self.alpha,r)

    if self.next_step(s,a) == '22':
      total_reward += 1000
    
    # total_reward = w1 * reward + w2 * t_reward
    return float(total_reward)
    

  def Reward_features(self,s,a):
    reward = 0
    t_reward = 0
    # ns = self.n_step
    w1 =1
    w2 =1

    if a == 'T':
      t_reward += -200
    else:
      reward += (-1) * self.distance[(s, self.next_step(s,a))]

    return (reward, t_reward)



env_2 = Environment2(data,distance)

In [ ]:
# @title Epsilon_greedy_ function
def epsilon_greedy(qvalue,state):
  epsilon = 0.4
  e =np.random.random()

  if e > epsilon:
    action = max(qvalue[state],key = qvalue[state].get)
  else :
    action = np.random.choice(list(qvalue[state].keys()))

  return action


def greedy(qvalue,state):

  action = max(qvalue[state],key = qvalue[state].get)
  return action


In [ ]:
# @title Agent Original
class Agent():
  def __init__(self, environment):
    self.envv = environment
    self.state = None
  
    self.q = {'1':{'F':0,'F2':0}, '2':{'F':0, 'B':0, 'T':0}, '3':{'F':0, 'B':0, 'T':0}, '4':{'B':0,'F':0},
        '21':{'F':0, 'B2':0}, '22':{'F':0, 'B':0, 'T':0}, '23':{'F':0, 'B':0, 'T':0}, '24':{'B':0}}
    
    self.p = {'1':'F', '2':'F', '3':'F', '4':'B',
        '21':'F', '22':'F', '23':'F', '24':'B'}

  def select_action(self,s):
    p_actions = self.envv.Possible_actions(s)
    action = epsilon_greedy(self.q,s)

    return action

  def select_greedy_action(self,s):
    p_actions = self.envv.Possible_actions(s)
    action = greedy(self.q,s)

    return action

  def observe(self,s):
    self.state = s
    self.a = self.select_action(s)
    reward = self.envv.Reward(self.state, self.a)
    g = 0.99
    next_s = self.envv.next_step(self.state,self.a)
    next_a = self.select_greedy_action(next_s)
    self._td_error = reward + (g * (self.q[next_s][next_a] - self.q[self.state][self.a]))
    
    return self._td_error

  def update(self):

    ## step_size = 0.1
    self.q[self.state][self.a] += int(0.1 * self._td_error)

    self.state = self.envv.next_step(self.state,self.a)
    return self.state
  
  def qvalue(self):
    return self.q
  
  def policy(self):
    for i in self.q.keys():
      self.p[i] = self.select_greedy_action(i)
    return self.p

  
agent = Agent(env)
agent_2 = Agent(env_2)

In [ ]:
# @title Loop Agent_1
def loop(environment, agent, initial_state, goal_state):
  
  episode = 0
  while episode <= 1000:
    step = 0
    reward_t = 0
    state = '1'
    goal = goal_state
    route =['1']

    while state != goal:

      agent.select_action(state)
      reward_t += agent.observe(state)
      state = agent.update()
      step += 1
      route.append(state)

    episode += 1
 

  return route

loop(env,agent,'1','24')
print(agent.qvalue())
print(agent.policy())
## loop for agent_2
loop(env_2,agent_2,'1','22')
print(agent_2.qvalue())
print(agent_2.policy())

Route policy function

In [ ]:
def route_policy(a,b,policy_,environment):
  route = [a]
  s = a
  while s != b:
    s = environment.next_step(s,policy_[s])
    route.append(s)
  return route

In [ ]:
print(route_policy('1','24',agent.policy(),env))
p = {'1':'F', '2':'F', '3':'F', '4':'F',
        '21':'F', '22':'F', '23':'F', '24':'B'}
print(route_policy('1','24', p, env))

p2 = {'1':'F', '2':'T', '3':'F', '4':'F',
        '21':'F', '22':'F', '23':'F', '24':'B'}
print(route_policy('1','22', agent_2.policy(), env_2))
print(route_policy('1','22', p2, env_2))

In [ ]:
env.alpha = [3,8]
agent = Agent(env)
loop(env,agent,'1','24')
print(route_policy('1','24',agent.policy(),env))
p = {'1':'F', '2':'F', '3':'F', '4':'F',
        '21':'F', '22':'F', '23':'F', '24':'B'}
print(route_policy('1','24', p, env))

Value function generator

In [ ]:
def valuefunctiongenerator(alpha_vec, policy_,environment_, origin, destination):  
  v={'d':0, 't':0}
  route = route_policy(origin,destination, policy_, environment_)
  
  i=1
  for s in route[0:-1]:
    (d,t) = environment_.Reward_features(s,policy_[s])

    v['d'] +=  d
    v['t'] +=  t
    i += 1

  V_basis = np.array([v['d'],v['t']])
  #alpha = [1,1]
  V = np.dot(alpha_vec, V_basis)

  return V, V_basis


In [ ]:
## Agent_1
print(valuefunctiongenerator([1,1],agent.policy(),env, '1', '24'))
print(valuefunctiongenerator([1,1],p,env,'1', '24'))
## Agent_2
print(valuefunctiongenerator([1,1],agent_2.policy(),env_2, '1', '22'))
print(valuefunctiongenerator([1,1],p2,env_2,'1', '22'))

# Inverse reinforcment learning (MaxEntropy approach)

In [ ]:
def IRL_MaxEnt(alpha_vec, expert_policy, expert_policy_2):
  learning_rate = 0.01
  V_input = np.zeros(2)
  penalty_factor = 2
  env.alpha = alpha_vec
  ## valufunction of expert policy for agent_1
  # V_expert, V_basis_expert = valuefunctiongenerator(alpha_vec, expert_policy, env, '1', '24')
  ## valufunction of expert policy for agent_2
  # V_expert_2, V_basis_expert_2 = valuefunctiongenerator(alpha_vec, expert_policy_2, env_2, '1', '22')
  
  npp=0
  n=1000
  pbar = tqdm(desc = 'while loop',total = n,position=0, leave=True)
  while npp < n:
    # print(npp)

    if npp ==0 :
      rl_policy = {'1':'F', '2':'T', '3':'T', '4':'F',
          '21':'F', '22':'F', '23':'F', '24':'B'}
      rl_policy_2 = {'1':'F2', '2':'T', '3':'T', '4':'F',
          '21':'F', '22':'F', '23':'F', '24':'B'}
    
    ## value function of RL policy
    ## Agent_1
    V_rl, V_basis_rl = valuefunctiongenerator(alpha_vec, rl_policy, env, '1', '24')
    ## Agent_2
    V_rl_2, V_basis_rl_2 = valuefunctiongenerator(alpha_vec, rl_policy_2, env_2, '1', '22')

    ## valufunction of expert policy for agent_1
    V_expert, V_basis_expert = valuefunctiongenerator(alpha_vec, expert_policy, env, '1', '24')
    ## valufunction of expert policy for agent_2
    V_expert_2, V_basis_expert_2 = valuefunctiongenerator(alpha_vec, expert_policy_2, env_2, '1', '22')
    

    if V_expert - V_rl >= 0:
      o=0
      if V_expert_2 - V_rl_2 < 0:
        learning_rate = 0.001
        gradient = V_basis_expert_2 - V_basis_rl_2
        delta = learning_rate * gradient
      else:
        print('Done')
        break

    else:
      gradient = V_basis_expert - V_basis_rl
      delta = learning_rate * gradient
    ## optimization problem  
    #res = scipy.optimize.linprog(V_input, bounds=(-1,1), method="simplex")
    #print('V_input',V_input)
    #res = linprog(V_input, bounds=(0,1), method="simplex")
    #res = linprog(V_input.tolist(), A_ub=[[V_input[0],V_input[1]]], b_ub=0, bounds=(0,1), method="simplex")
    if alpha_vec[0] + delta[0] < 0:
      delta[0] = 0 - alpha_vec[0]/2

    alpha_vec_new = alpha_vec + delta
    env.alpha = alpha_vec_new
    env_2.alpha = alpha_vec_new
    alpha_vec = alpha_vec_new
    #run the RL for finding the new policy
    agent = Agent(env)
    loop(env,agent,'1','24')
    rl_policy = agent.policy()

  

    agent_2 = Agent(env_2)
    loop(env_2,agent_2,'1','22')
    rl_policy_2 = agent_2.policy()
    
    #alpha_vec_new = res.x
    #V_expert_new = np.dot(alpha_vec_new, V_basis_expert)
    npp += 1
    pbar.update(1)


  print(agent.policy())
  print(route_policy('1','24',agent.policy(),env))
  print(env.alpha)
  print(agent_2.policy())
  print(route_policy('1','22',agent_2.policy(),env_2))
  print(env_2.alpha)
  pbar.close()

In [ ]:
IRL_MaxEnt([5,5], p, p2)

while loop:  10%|▉         | 96/1000 [00:30<04:51,  3.10it/s]

Done
{'1': 'F', '2': 'F', '3': 'F', '4': 'F', '21': 'F', '22': 'F', '23': 'F', '24': 'B'}
['1', '2', '3', '4', '24']
[2.57 6.8 ]
{'1': 'F', '2': 'T', '3': 'B', '4': 'B', '21': 'F', '22': 'F', '23': 'B', '24': 'B'}
['1', '2', '22']
[2.57 6.8 ]


In [ ]:
#@title IRL_linear programming
def IRL_LP(alpha_vec, expert_policy):

  V_input = np.zeros(2)
  penalty_factor = 2
  ## valufunction of expert policy
  V_expert, V_basis_expert = valuefunctiongenerator(alpha_vec, expert_policy, env)
  
  npp=0
  while npp < 2:
    
    if npp ==0 :
      rl_policy = p = {'1':'F', '2':'F', '3':'T', '4':'B',
          '21':'F', '22':'F', '23':'F', '24':'B'}
    
    ## value function of RL policy
    V_rl, V_basis_rl = valuefunctiongenerator(alpha_vec, rl_policy, env)

    # print(V_basis_rl,V_basis_expert)
    ## linaer programming
    if V_expert - V_rl >= 0:
      V_input += V_basis_rl - V_basis_expert
    else:
      V_input += penalty_factor* (V_basis_rl - V_basis_expert)
    ## optimization problem  
    #res = scipy.optimize.linprog(V_input, bounds=(-1,1), method="simplex")
    print('V_input',V_input)
    #res = linprog(V_input, bounds=(0,1), method="simplex")
    #res = linprog(V_input.tolist(), A_ub=[[V_input[0],V_input[1]]], b_ub=0, bounds=(0,1), method="simplex")

    #alpha_vec_new = res.x
    V_expert_new = np.dot(alpha_vec_new, V_basis_expert)
    npp += 1
    # print(res)
    # print(res.x)
